In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids, max_length = 100)
print(tokenizer.decode(outputs[0]))

In [ ]:
input_text = "What is the date today? Today 9 Mart"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
from googlesearch import search
import requests
import re
from bs4 import BeautifulSoup

bs = BeautifulSoup(response.text,"lxml")

# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext


query = "What is the date today?"

for i in search(query, tld="co.in", num=1, stop=1, pause=2):
    url = str(i)
    response = requests.get(url)
    e = response.text

    print(cleanhtml(response.text))

In [21]:
import requests
from duckduckgo_search import DDGS

query = "what is mouse?"

r = requests.get("https://api.duckduckgo.com",
    params = {
        "q": query,
        "format": "json"
    })


def not_plash(query):
  results = DDGS().text(query, max_results=1)
  return list(map(lambda x: x['body'], results))

data = r.json()

# print(data)
if len(data["Abstract"]) != 0:
  print(data["Abstract"])
elif len(data["Answer"]) != 0:
  print(data["Answer"])
else:
  print(*not_plash(query))

A mouse is a small rodent. Characteristically, mice are known to have a pointed snout, small rounded ears, a body-length scaly tail, and a high breeding rate. The best known mouse species is the common house mouse. Mice are also popular as pets. In some places, certain kinds of field mice are locally common. They are known to invade homes for food and shelter. Mice are typically distinguished from rats by their size. Generally, when a muroid rodent is discovered, its common name includes the term mouse if it is smaller, or rat if it is larger. The common terms rat and mouse are not taxonomically specific. Typical mice are classified in the genus Mus, but the term mouse is not confined to members of Mus and can also apply to species from other genera such as the deer mouse. Domestic mice sold as pets often differ substantially in size from the common house mouse. This is attributable to breeding and different conditions in the wild. The best-known strain of mouse is the white lab mouse.

In [16]:
import requests
from duckduckgo_search import DDGS
query = "Why is the grass green?"

def not_plash(query):
  results = DDGS().text(query, max_results=1)
  return list(map(lambda x: x['body'], results))

print(*not_plash(query))

Grass is green because of the presence of chlorophyll throughout the leaves and stems. Chlorophyll is a pigment that allows plants to capture energy from the Sun in order to power the process of photosynthesis. Given that most grasses lack trunks or woody stems like trees or shrubs, their aboveground bodies are mostly chlorophyll-rich leaves ...


In [4]:
from duckduckgo_search import DDGS, AsyncDDGS


# DDG query wrapper
def results(query: str, n_max=1, include_urls=False) -> list:
    results = DDGS().text(query, max_results=n_max)
    if include_urls:
        return [(e['body'], e['href']) for e in results]
    return [e['body'] for e in results]

# # Whatever this is
# async def async_results(query: str, n_max=1, include_urls=False) -> list:
#     results = await AsyncDDGS().text(query, max_results=n_max)
#     if include_urls:
#         return [(e['body'], e['href']) for e in results]
#     return [e['body'] for e in results]

# Unwrap single result or return respone_missing
def result_single(query: str) -> str:
    tmp = results(query)
    if len(tmp):
        return tmp[0]
    return None


if __name__ == '__main__':
    COL_GRN = '\033[0;32m'
    COL_RED = '\033[0;31m'
    COL_RESET = '\033[m'
    print(f'{COL_RED}[A]{COL_RESET}: {result_single(input(f"{COL_GRN}[Q]{COL_RESET}: "))}')


[Q]: What are the latest football scores in the Premier League?
[A]: Premier League scores, results and fixtures on BBC Sport, including live football scores, goals and goal scorers.


In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# load jsonl dataset
dataset = load_dataset("json", data_files="out.jsonl", split="train")

training_args = TrainingArguments(save_strategy = 'epoch', output_dir = 'train_outputs')

trainer = SFTTrainer(
    "openai-community/gpt2",
    args=training_args,
    train_dataset=dataset,
    packing=True,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:161: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [8]:
import json

output = '''{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What`s the capital of France?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "..."}]}'''

output = output * 100


with open('out.jsonl', 'w') as f:
    f.write(output)

